In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import pandas as pd
from archaeo_super_prompt.dataset.normalization.intervention_date import utils, transforms
from archaeo_super_prompt.dataset.normalization.intervention_date import duration


pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv(Path("~/Documents/Mappa/ressources/interventi_thomas.csv"),
                 sep=";").drop(columns=["geojson", "centroid", "raster_wms", "matrix", "geom_edit"]).astype({"anno": pd.Int32Dtype()})
print("Records number:", len(df))

## Normalization of the duration

### Duration data to keep

For the training and the evaluation of a model predicting the duration of an excavation, only keep the records which were inserted during the `Progetto MAGOH`. In fact, in these records, the `durata_intervento` value was empty only when the information was not available the documents. So, with these records, we can teach to the model when to answer "I don't know".

In [ ]:
df["motivazione_aggiornamento"].unique()

In [ ]:
normalized = df.assign(
    norm_duration=df["durata_intervento"].apply(
        lambda k: None if isinstance(k, float) else k
    ).apply(duration.parse_duration)
)
normalized = normalized[
    normalized["norm_duration"].isnull() |
    normalized["norm_duration"].apply(lambda k: isinstance(k, tuple))
    ]
print("Records number:", len(df))

## Normalization of the intervention start date

We want to store a digital estimation of the start of an intervention with the form of a time interval.

In [ ]:
gettable_data = normalized[[
    "idscheda", "data_protocollo", "data_intervento", "anno", "norm_duration"
]]
gettable_data = gettable_data[
    # (gettable_data["data_protocollo"].notnull()) &
    (gettable_data["data_intervento"].notnull())
    & (gettable_data["anno"].notnull())
    ]
len(gettable_data)

In [ ]:
normalized = utils.pipe(gettable_data, (
    transforms.generic_period,
    transforms.generic_single_period,
    transforms.precised_numeric_start_date,
    transforms.before_day_month,
))
is_date_processed = normalized["norm_date"].apply(
    lambda s: s is not None
)
print("covered_dates:", sum(is_date_processed)/len(is_date_processed)*100, "%")
normalized[~is_date_processed]

In [ ]:
for i, d in enumerate(
    normalized[~is_date_processed]["data_intervento"].unique()
):
    print(d)

In [ ]:
normalized = normalized[is_date_processed]
print("Records number:", len(normalized))

### Pre-normalized date to digital datetime

After this normalization, the column `norm_date` is a tuple with the following elements:
- a date string or a the `<UNKNOWN>` string when we cannot give a starting date to the intervention
- a date string for the most recent date before which the intervention could have started
- a precision string with the value `day`, `month`, `year`

The date string has thist format for now: `d/m/y`, with `d` a one-or-two-digits integer for the day, `y` a 4-digit integer for the year and `m` a string for the month to be normalized into a digit between 1 and 12.

**The aim of this section is to convert the date strings into a processable datetime object.**

In [ ]:
from datetime import date as ddate
from archaeo_super_prompt.dataset.normalization.intervention_date import month_normalization
from logging import warning

bad_words = set()

def to_datetime(date):
    if date == "<UNKNOWN>":
        return None
    d, m, y = date.split("/")
    m = month_normalization.to_int_month(m)
    return ddate(int(y), m, int(d))

normalized = normalized.assign(
    start_date=lambda df: df["norm_date"].apply(lambda nd: to_datetime(nd.start_date)),
    end_date=lambda df: df["norm_date"].apply(lambda nd: to_datetime(nd.end_date)),
    precision=lambda df: df["norm_date"].apply(lambda nd: nd.precision)
)

### Fix human insertion noise

The got duration and date of intervention estimations are not always representing both these fields.

Indeed, instead of inputing a time interval for the estimation of the start of the intervention, a period has usually been inserted to represent the whole period of the intervention.
In this case, we do the following:
- we only keep the earlier date as the start date of the intervention
- with the initial couple (`start_date` - `end_date`), we infer the duration, if not input.

We apply this fix for each got pre-processed record.

In [ ]:
from archaeo_super_prompt.dataset.normalization.intervention_date import period_to_dd
from datetime import timedelta

normalized = period_to_dd.fix_start_and_duration(normalized)
print("Records number:", len(normalized))

## Final dataframe

We export a new training set with the window of intervention start

### Renaming

In [ ]:
complete_table = normalized.rename(columns={
    "start_date": "intervention_start_date_min",
    "end_date": "intervention_start_date_max",
    "precision": "intervention_start_date_precision"
    }).assign(
    duration_value=normalized["norm_duration"].apply(
        lambda d: None if d is None else d[0],
    ),
    duration_precision=normalized["norm_duration"].apply(
        lambda d: None if d is None else d[1],
    )
)[[
    "idscheda",
    "intervention_start_date_min",
    "intervention_start_date_max",
    "intervention_start_date_precision",
    "duration_value",
    "duration_precision"
]]
print("Normalized record number:", len(complete_table))
complete_table

In [ ]:
from archaeo_super_prompt.dataset.postgresql_engine import get_entries

In [ ]:
it, findings = get_entries(10000, 0.1, False)

In [ ]:
complete_table = complete_table.merge(df, on="idscheda").merge(
    it, left_on="idscheda", right_on="scheda_intervento.id"
)

In [ ]:
complete_table

In [ ]:
from archaeo_super_prompt.config.env import getenv_or_throw
from sqlalchemy import create_engine

def _create_engine_from_credentials():
    DIALECT = "postgresql"
    DRIVER = "psycopg2"
    writing_db_user = getenv_or_throw("PG_SUPERUSER")
    db_name = getenv_or_throw("PG_DB_NAME")
    db_user_password = getenv_or_throw("PG_DB_PASSWORD")

    db_host = getenv_or_throw("PG_DB_HOST")
    db_port = getenv_or_throw("PG_DB_PORT")

    return create_engine(
        f"{DIALECT}+{DRIVER}://{writing_db_user}:{db_user_password}@{db_host}:{db_port}/{db_name}"
    )
connection = _create_engine_from_credentials()

In [ ]:
complete_table.to_sql("featured__intervention_data", connection)